In [1]:
from __future__ import annotations

In [2]:
from functools import partial

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torchvision.ops.misc import Conv2dNormActivation

from typing import NamedTuple

In [4]:
hl_fm = torch.randn(size=(1, 512, 13, 13))
ml_fm = torch.randn(size=(1, 256, 26, 26))

In [6]:
torch.add(hl_fm, ml_fm)

RuntimeError: The size of tensor a (13) must match the size of tensor b (26) at non-singleton dimension 3

In [7]:
hl_fm = torch.randn(size=(1, 512, 26, 26))
ml_fm = torch.randn(size=(1, 256, 26, 26))

torch.add(hl_fm, ml_fm)

RuntimeError: The size of tensor a (512) must match the size of tensor b (256) at non-singleton dimension 1

In [8]:
from functools import partial
from torchvision.ops.misc import Conv2dNormActivation

LeakyRelu_Inplace = partial(
    nn.LeakyReLU,
    negative_slope=0.1,
    inplace=True,
)
ConvBlockReduceChannels = partial(Conv2dNormActivation,
                                  kernel_size=1,
                                  activation_layer=LeakyRelu_Inplace)

In [9]:
hl_fm = torch.randn(size=(1, 512, 13, 13))
ml_fm = torch.randn(size=(1, 256, 26, 26))
ll_fm = torch.randn(size=(1, 128, 52, 52))

In [26]:
conv_hl_reduce = ConvBlockReduceChannels(in_channels=512, out_channels=64)
conv_ml_reduce = ConvBlockReduceChannels(in_channels=256, out_channels=64)
conv_ll_reduce = ConvBlockReduceChannels(in_channels=128, out_channels=64)

In [11]:
hl_fm_r = conv_hl_reduce(hl_fm)
ml_fm_r = conv_ml_reduce(ml_fm)
ll_fm_r = conv_ll_reduce(ll_fm)

In [12]:
print(f"New HL shape - {hl_fm_r.shape}")
print(f"New ML shape - {ml_fm_r.shape}")
print(f"New LL shape - {ll_fm_r.shape}")

New HL shape - torch.Size([1, 64, 13, 13])
New ML shape - torch.Size([1, 64, 26, 26])
New LL shape - torch.Size([1, 64, 52, 52])


In [27]:
hl_upsampler = nn.Upsample(scale_factor=2, mode="nearest")

In [14]:
hl_fm_r_upsampled = hl_upsampler(hl_fm_r)

hl_fm_r_upsampled.shape

torch.Size([1, 64, 26, 26])

In [15]:
hl_ml_fused = torch.add(hl_fm_r_upsampled, ml_fm_r)

hl_ml_fused.shape

torch.Size([1, 64, 26, 26])

In [16]:
ConvSmoother = partial(
                Conv2dNormActivation,
                  in_channels=64,
                  out_channels=64,
                  kernel_size=3,
                  activation_layer=LeakyRelu_Inplace
               )

hl_ml_fused_smoother = ConvSmoother()

smooth_hl_ml_fused = hl_ml_fused_smoother(hl_ml_fused)

print(smooth_hl_ml_fused.shape)

torch.Size([1, 64, 26, 26])


In [28]:
class FakeBackboneResult(NamedTuple):
  hl_features: torch.Tensor
  ml_features: torch.Tensor
  ll_features: torch.Tensor

In [18]:
class FakeBackbone(nn.Module):
  def __init__(self):
    super().__init__()

  def forward(self, x: torch.Tensor) -> FakeBackboneResult:
    hl_fm = torch.randn(size=(1, 512, 13, 13))
    ml_fm = torch.randn(size=(1, 256, 26, 26))
    ll_fm = torch.randn(size=(1, 128, 52, 52))

    return FakeBackboneResult(
        hl_features=hl_fm,
        ml_features=ml_fm,
        ll_features=ll_fm
    )

In [19]:
class FPNNeck(nn.Module):
  def __init__(self, in_channels: list[int], out_channels: int):
    super().__init__()

    LeakyRelu_Inplace = partial(
        nn.LeakyReLU,
        negative_slope=0.1,
        inplace=True,
    )

    ConvBlock = partial(Conv2dNormActivation, activation_layer=LeakyRelu_Inplace)

    self.hl_channel_reducer = ConvBlock(
        in_channels=in_channels[0],
        out_channels=out_channels,
        kernel_size=1,
    )

    self.ml_channel_reducer = ConvBlock(
        in_channels=in_channels[1],
        out_channels=out_channels,
        kernel_size=1,
    )

    self.ll_channel_reducer = ConvBlock(
        in_channels=in_channels[2],
        out_channels=out_channels,
        kernel_size=1,
    )

    self.ml_smoother = ConvBlock(
        in_channels=out_channels,
        out_channels=out_channels,
    )

    self.ll_smoother = ConvBlock(
        in_channels=out_channels,
        out_channels=out_channels,
    )

  def forward(self,
              hl_features:torch.Tensor,
              ml_features:torch.Tensor,
              ll_features:torch.Tensor):

    hl_channel_r = self.hl_channel_reducer(hl_features)
    ml_channel_r = self.ml_channel_reducer(ml_features)
    ll_channel_r = self.ll_channel_reducer(ll_features)

    upsample_hl = F.interpolate(
        hl_channel_r,
        size=[ml_channel_r.size(2), ml_channel_r.size(3)],
        mode="nearest",
    )

    fused_hl_ml = upsample_hl + ml_channel_r
    smoothed_ml_features = self.ml_smoother(fused_hl_ml)

    upsample_ml = F.interpolate(
        smoothed_ml_features,
        size=[ll_channel_r.size(2), ll_channel_r.size(3)],
        mode="nearest",
    )
    fused_ml_ll = upsample_ml + ll_channel_r
    smoothed_ll_features = self.ll_smoother(fused_ml_ll)

    out = [hl_channel_r, smoothed_ml_features, smoothed_ll_features]
    return out


In [20]:
neck = FPNNeck(in_channels=[512, 256, 128], out_channels=64)

In [21]:
backbone = FakeBackbone()

backbone_output = backbone(torch.randn(size=(1, 3, 416, 416)))

In [22]:
enriched_hl_features, enriched_ml_features, enriched_ll_features = neck(backbone_output.hl_features,
     backbone_output.ml_features,
     backbone_output.ll_features)

In [23]:
enriched_hl_features.shape

torch.Size([1, 64, 13, 13])

In [24]:
enriched_ml_features.shape

torch.Size([1, 64, 26, 26])

In [25]:
enriched_ll_features.shape

torch.Size([1, 64, 52, 52])